In [1]:
import pandas as pd
import xgboost as xgb

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 

In [3]:
datasetTitanic = pd.read_csv("train.csv")
datasetTitanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Show row/column data info
datasetTitanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# How many null values has each column
datasetTitanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
# Change null values of 'Age' column to median so they are not empty
datasetTitanic['Age'].fillna(datasetTitanic['Age'].median(), inplace = True)

In [7]:
# Change null values of 'Embarked' column to value most repeated, so they are not empty
datasetTitanic['Embarked'].fillna(datasetTitanic['Embarked'].mode()[0], inplace = True)

In [8]:
# Too much null values in 'Cabin' column, so its deleted 
datasetTitanic.drop('Cabin', axis = 1, inplace = True)

In [9]:
# Convert categorical values of the variable 'Sex' to numeric
datasetTitanic['Sex'] = datasetTitanic['Sex'].map({'female': 0, 'male': 1})

In [10]:
# Normalize 'Age' variable
mean_age = datasetTitanic['Age'].mean()
std_age = datasetTitanic['Age'].std()
datasetTitanic['Age'] = (datasetTitanic['Age'] - mean_age) / std_age

In [11]:
# Delete this 3 columns because they are irrelevant
datasetTitanic.drop(['Name', 'Ticket', 'Embarked'], axis = 1, inplace = True)

In [12]:
# Save the data in another different .CSV file after correct preprocessing and cleaning
datasetTitanic.to_csv("data_cleansing.csv")

In [13]:
X = datasetTitanic.drop('Survived', axis=1)
y = datasetTitanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Classification report:\n", classification_report(y_test, y_pred))

Classification report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       117
           1       0.77      0.71      0.74        62

    accuracy                           0.83       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.82      0.83      0.82       179



In [14]:
# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[104  13]
 [ 18  44]]


In [15]:
datasetTitanicMinimized = datasetTitanic[['PassengerId', 'Survived']]
print(datasetTitanicMinimized)

     PassengerId  Survived
0              1         0
1              2         1
2              3         1
3              4         1
4              5         0
..           ...       ...
886          887         0
887          888         1
888          889         0
889          890         1
890          891         0

[891 rows x 2 columns]


In [16]:
# The dataset is traversed, with the PassengerId and Survived columns, to see how many people died and how many survived.
survivors = []
deaths = []
for i in datasetTitanicMinimized['Survived']:
    if i == 0:
        survivors.append(i)
        
    elif i == 1:
        deaths.append(i)
        
alive = len(survivors)
print("Number of people who survived: ", alive)

dead = len(deaths)
print("Number of people who died: ", dead)

print("Total people on board the Titanic: ", dead + alive)

Number of people who survived:  549
Number of people who died:  342
Total people on board the Titanic:  891


In [17]:
# Save the dataset in a different .CSV file with only a couple of columns: PassengerId and Survived
datasetTitanicMinimized.to_csv("Titanic_Passenger_Survived.csv")